In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rsna-pneumonia-processed-dataset/stage2_train_metadata.csv
/kaggle/input/rsna-pneumonia-processed-dataset/stage2_test_metadata.csv


In [4]:
import os
DATA_DIR = "/kaggle/input/rsna-pneumonia-processed-dataset"  # adjust if needed
print(os.listdir(DATA_DIR))


['stage2_train_metadata.csv', 'stage2_test_metadata.csv', 'Training', 'Test']


In [4]:
import os

DATA_DIR = "/kaggle/input/rsna-pneumonia-processed-dataset"

print("Top level:")
print(os.listdir(DATA_DIR))

print("\nTraining folder (first 5):")
print(os.listdir(f"{DATA_DIR}/Training")[:5])

print("\nTest folder (first 5):")
print(os.listdir(f"{DATA_DIR}/Test")[:5])


Top level:
['stage2_train_metadata.csv', 'stage2_test_metadata.csv', 'Training', 'Test']

Training folder (first 5):
['Images', 'Masks']

Test folder (first 5):
['1b66ef47-b795-418f-bc18-1339bc980b8b.png', '1f69bd0a-8e25-4c4e-b5be-f4395f032f58.png', '29d959bb-171d-487d-87b3-8637e30ba480.png', 'c00a132f-6108-46e7-a67a-77efd3d2d0ef.png', '2aa6506f-a7a0-4057-bc65-e56a27370698.png']


In [5]:
import pandas as pd
import os

DATA_DIR = "/kaggle/input/rsna-pneumonia-processed-dataset"

train_df = pd.read_csv(f"{DATA_DIR}/stage2_train_metadata.csv")

print("Original class distribution:")
print(train_df["class"].value_counts())


Original class distribution:
class
No Lung Opacity / Not Normal    11821
Lung Opacity                     9555
Normal                           8851
Name: count, dtype: int64


In [6]:
# Keep only NORMAL and LUNG OPACITY
clean_df = train_df[train_df["class"].isin(["Normal", "Lung Opacity"])].copy()

# Map labels
clean_df["label"] = clean_df["class"].map({
    "Normal": 0,
    "Lung Opacity": 1
})

print("\nClean class distribution:")
print(clean_df["label"].value_counts())



Clean class distribution:
label
1    9555
0    8851
Name: count, dtype: int64


In [8]:
IMAGE_DIR = f"{DATA_DIR}/Training/Images"

clean_df["image_path"] = clean_df["patientId"].apply(
    lambda x: os.path.join(IMAGE_DIR, f"{x}.png")
)

# Remove rows where image file missing
clean_df = clean_df[clean_df["image_path"].apply(os.path.exists)]

print("\nFinal usable samples:", len(clean_df))



Final usable samples: 18406


In [9]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    clean_df,
    test_size=0.15,
    stratify=clean_df["label"],
    random_state=42
)

print("Train size:", len(train_df))
print("Val size:", len(val_df))

print("\nTrain distribution:")
print(train_df["label"].value_counts())

print("\nVal distribution:")
print(val_df["label"].value_counts())


Train size: 15645
Val size: 2761

Train distribution:
label
1    8122
0    7523
Name: count, dtype: int64

Val distribution:
label
1    1433
0    1328
Name: count, dtype: int64


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 300
BATCH_SIZE = 16


2026-02-11 07:00:46.134588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770793246.326910      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770793246.385425      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770793246.847136      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770793246.847181      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770793246.847184      55 computation_placer.cc:177] computation placer alr

In [11]:
train_df["label"] = train_df["label"].astype(str)
val_df["label"] = val_df["label"].astype(str)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=7,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image_path",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="binary",
    batch_size=BATCH_SIZE
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="image_path",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="binary",
    batch_size=BATCH_SIZE
)


Found 15645 validated image filenames belonging to 2 classes.
Found 2761 validated image filenames belonging to 2 classes.


In [12]:
train_df["label"].value_counts()


label
1    8122
0    7523
Name: count, dtype: int64

In [21]:
IMG_SIZE = 300
BATCH_SIZE = 16

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=7,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image_path",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="binary",
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="image_path",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="binary",
    batch_size=BATCH_SIZE,
    shuffle=False
)


Found 15645 validated image filenames belonging to 2 classes.
Found 2761 validated image filenames belonging to 2 classes.


In [14]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def se_block(input_tensor, ratio=16):
    filters = input_tensor.shape[-1]

    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // ratio, activation="relu")(se)
    se = Dense(filters, activation="sigmoid")(se)
    se = Reshape((1, 1, filters))(se)

    return Multiply()([input_tensor, se])


In [15]:
from tensorflow.keras.applications import EfficientNetB3

base_model = EfficientNetB3(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False  

x = base_model.output

x = se_block(x)

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)

model.summary()


I0000 00:00:1770793765.944739      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1770793765.950489      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 300, 300,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 300, 300,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 300, 300,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │      1,080 │ stem_conv_pad[0]… │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        160 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        360 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        160 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 40)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 40)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 10)  │        410 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 40)  │        440 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 40)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        960 │ block1a_se_excit

 Total params: 11,479,952 (43.79 MB)

 Trainable params: 693,345 (2.64 MB)

 Non-trainable params: 10,786,607 (41.15 MB)

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)


In [19]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.3),
    ModelCheckpoint("best_effb3_se.keras", save_best_only=True)
]


In [22]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 472s 482ms/step - accuracy: 0.8385 - auc: 0.9118 - loss: 0.3807 - precision_1: 0.8512 - recall_1: 0.8327 - val_accuracy: 0.8982 - val_auc: 0.9653 - val_loss: 0.2547 - val_precision_1: 0.9299 - val_recall_1: 0.8695 - learning_rate: 1.0000e-04
Epoch 2/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 476s 487ms/step - accuracy: 0.8907 - auc: 0.9534 - loss: 0.2763 - precision_1: 0.8979 - recall_1: 0.8902 - val_accuracy: 0.9160 - val_auc: 0.9724 - val_loss: 0.2171 - val_precision_1: 0.9451 - val_recall_1: 0.8897 - learning_rate: 1.0000e-04
Epoch 3/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 476s 486ms/step - accuracy: 0.8950 - auc: 0.9571 - loss: 0.2635 - precision_1: 0.9134 - recall_1: 0.8854 - val_accuracy: 0.9145 - val_auc: 0.9735 - val_loss: 0.2183 - val_precision_1: 0.9497 - val_recall_1: 0.8821 - learning_rate: 1.0000e-04
Epoch 4/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 472s 483ms/step - accuracy: 0.9031 - auc: 0.9607 - loss: 0.2506 - precision_1: 0.9187 - recall_1: 0.8918 - val_acc

In [23]:
base_model.trainable = True

for layer in base_model.layers[:-60]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10


2026-02-11 09:14:58.595771: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:14:58.742819: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:14:59.075393: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:14:59.224575: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:14:59.371998: E external/local_xla/xla/stream_

265/978 ━━━━━━━━━━━━━━━━━━━━ 5:32 466ms/step - accuracy: 0.8442 - auc: 0.9277 - loss: 0.3539 - precision_2: 0.8282 - recall_2: 0.8864

2026-02-11 09:17:28.190734: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:17:28.339893: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:17:28.650618: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:17:28.798788: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 09:17:29.129402: E external/local_xla/xla/stream_

978/978 ━━━━━━━━━━━━━━━━━━━━ 581s 543ms/step - accuracy: 0.8680 - auc: 0.9418 - loss: 0.3140 - precision_2: 0.8597 - recall_2: 0.8922 - val_accuracy: 0.9178 - val_auc: 0.9732 - val_loss: 0.2192 - val_precision_2: 0.9389 - val_recall_2: 0.9002 - learning_rate: 1.0000e-05
Epoch 2/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 482s 493ms/step - accuracy: 0.9061 - auc: 0.9634 - loss: 0.2429 - precision_2: 0.9175 - recall_2: 0.8997 - val_accuracy: 0.9229 - val_auc: 0.9770 - val_loss: 0.2031 - val_precision_2: 0.9446 - val_recall_2: 0.9044 - learning_rate: 1.0000e-05
Epoch 3/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 487s 498ms/step - accuracy: 0.9072 - auc: 0.9670 - loss: 0.2314 - precision_2: 0.9176 - recall_2: 0.9023 - val_accuracy: 0.9283 - val_auc: 0.9797 - val_loss: 0.1911 - val_precision_2: 0.9571 - val_recall_2: 0.9023 - learning_rate: 1.0000e-05
Epoch 4/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 482s 493ms/step - accuracy: 0.9180 - auc: 0.9718 - loss: 0.2118 - precision_2: 0.9332 - recall_2: 0.9092 - val_accuracy: 0.93

In [24]:
model.save("final_effb3_se_rsna.keras")

In [25]:
import numpy as np
from sklearn.metrics import accuracy_score

val_gen.reset()
y_pred_prob = model.predict(val_gen)

best_acc = 0
best_thresh = 0

for t in np.arange(0.3, 0.7, 0.01):
    y_pred = (y_pred_prob > t).astype(int)
    acc = accuracy_score(val_gen.classes, y_pred)
    
    if acc > best_acc:
        best_acc = acc
        best_thresh = t

print("Best Threshold:", best_thresh)
print("Best Accuracy:", best_acc)


173/173 ━━━━━━━━━━━━━━━━━━━━ 55s 259ms/step
Best Threshold: 0.3
Best Accuracy: 0.9398768562115176


In [11]:
train_df["label"] = train_df["label"].astype(str)
val_df["label"] = val_df["label"].astype(str)
IMG_SIZE = 380
BATCH_SIZE = 8

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=7,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image_path",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="binary",
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="image_path",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="binary",
    batch_size=BATCH_SIZE,
    shuffle=False
)


Found 15645 validated image filenames belonging to 2 classes.
Found 2761 validated image filenames belonging to 2 classes.


In [12]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import tensorflow as tf

base_model = EfficientNetB4(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

model.summary()


I0000 00:00:1770819070.246329      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1770819070.252888      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 380, 380,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 380, 380,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 380, 380,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 380, 380,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 381, 381,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 190, 190,  │      1,296 │ stem_conv_pad[0]… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 190, 190,  │        192 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 190, 190,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 190, 190,  │        432 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 190, 190,  │        192 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 190, 190,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 48)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 48)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 12)  │        588 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 48)  │        624 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 190, 190,  │          0 │ block1a_activati… │
│ (Multiply)          │ 48)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 190, 190,  │      1,152 │ block1a_se_excit

 Total params: 18,599,520 (70.95 MB)

 Trainable params: 922,113 (3.52 MB)

 Non-trainable params: 17,677,407 (67.43 MB)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor="val_auc", mode="max", patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.3),
    ModelCheckpoint(
        "/kaggle/working/best_effb4.keras",
        monitor="val_auc",
        mode="max",
        save_best_only=True
    )
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks
)

# Force final save
model.save("/kaggle/working/final_effb4_rsna.keras")



Epoch 1/5


I0000 00:00:1770819194.280286     154 service.cc:152] XLA service 0x7d1c20002620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770819194.280327     154 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1770819194.280331     154 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1770819198.773447     154 cuda_dnn.cc:529] Loaded cuDNN version 91002
2026-02-11 14:13:28.612414: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:13:28.749498: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:13:28.984259: E external/local_xl

 848/1956 ━━━━━━━━━━━━━━━━━━━━ 7:04 383ms/step - accuracy: 0.8251 - auc: 0.8936 - loss: 0.4267 - precision: 0.8297 - recall: 0.8400

2026-02-11 14:19:22.818914: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:19:22.953529: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:19:23.180647: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:19:23.314872: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:19:23.558983: E external/local_xla/xla/stream_

1956/1956 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.8420 - auc: 0.9119 - loss: 0.3908 - precision: 0.8479 - recall: 0.8502

2026-02-11 14:27:51.057661: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:27:51.192291: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:27:51.504874: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:27:51.647985: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 14:27:52.373924: E external/local_xla/xla/stream_

1956/1956 ━━━━━━━━━━━━━━━━━━━━ 898s 437ms/step - accuracy: 0.8420 - auc: 0.9119 - loss: 0.3908 - precision: 0.8479 - recall: 0.8502 - val_accuracy: 0.9076 - val_auc: 0.9713 - val_loss: 0.2270 - val_precision: 0.9396 - val_recall: 0.8786 - learning_rate: 1.0000e-04
Epoch 2/5
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 637s 326ms/step - accuracy: 0.8805 - auc: 0.9476 - loss: 0.2993 - precision: 0.8899 - recall: 0.8813 - val_accuracy: 0.9156 - val_auc: 0.9704 - val_loss: 0.2284 - val_precision: 0.9304 - val_recall: 0.9051 - learning_rate: 1.0000e-04
Epoch 3/5
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 633s 324ms/step - accuracy: 0.8897 - auc: 0.9518 - loss: 0.2860 - precision: 0.8986 - recall: 0.8874 - val_accuracy: 0.9102 - val_auc: 0.9707 - val_loss: 0.2359 - val_precision: 0.9315 - val_recall: 0.8925 - learning_rate: 1.0000e-04
Epoch 4/5
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 637s 326ms/step - accuracy: 0.8934 - auc: 0.9592 - loss: 0.2604 - precision: 0.8998 - recall: 0.8947 - val_accuracy: 0.9181 - val_auc: 0.9749 - 

In [15]:
base_model.trainable = True

for layer in base_model.layers[:-100]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)

# Save again after fine-tuning
model.save("/kaggle/working/final_effb4_rsna_finetuned.keras")


Epoch 1/10


2026-02-11 15:16:12.860876: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:16:13.013453: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:16:13.212952: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:16:13.365051: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:16:13.723788: E external/local_xla/xla/stream_

 176/1956 ━━━━━━━━━━━━━━━━━━━━ 9:03 305ms/step - accuracy: 0.8219 - auc: 0.9157 - loss: 0.3799 - precision_1: 0.7938 - recall_1: 0.8794

2026-02-11 15:17:39.492055: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:17:39.644624: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:17:39.844108: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:17:39.995562: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-11 15:17:40.317854: E external/local_xla/xla/stream_

1956/1956 ━━━━━━━━━━━━━━━━━━━━ 755s 354ms/step - accuracy: 0.8686 - auc: 0.9398 - loss: 0.3235 - precision_1: 0.8653 - recall_1: 0.8858 - val_accuracy: 0.9207 - val_auc: 0.9780 - val_loss: 0.1930 - val_precision_1: 0.9299 - val_recall_1: 0.9163 - learning_rate: 1.0000e-05
Epoch 2/10
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 643s 329ms/step - accuracy: 0.8981 - auc: 0.9586 - loss: 0.2622 - precision_1: 0.9126 - recall_1: 0.8892 - val_accuracy: 0.9279 - val_auc: 0.9815 - val_loss: 0.1762 - val_precision_1: 0.9407 - val_recall_1: 0.9191 - learning_rate: 1.0000e-05
Epoch 3/10
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 640s 327ms/step - accuracy: 0.9104 - auc: 0.9656 - loss: 0.2380 - precision_1: 0.9190 - recall_1: 0.9075 - val_accuracy: 0.9355 - val_auc: 0.9827 - val_loss: 0.1690 - val_precision_1: 0.9524 - val_recall_1: 0.9218 - learning_rate: 1.0000e-05
Epoch 4/10
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 639s 327ms/step - accuracy: 0.9224 - auc: 0.9760 - loss: 0.1958 - precision_1: 0.9331 - recall_1: 0.9157 - val_accura

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/final_effb3_se_rsna.keras", compile=False)

model.compile(
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)
